Homework goal is to train a ride duration prediction model using data from NYC taxi dataset 2024.

ENV: I am using Github codespaces and Anaconda to set up my environment. Another option is to use AWS Linux env, but I would not be using that. 

### Dataset source

The New York and Limousine Commission and I am using the yellow taxi, use the dataset for January and February 2023
`https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet`

`https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet`

In [6]:
import pandas as pd # working with tabular data
import numpy as np
import pickle # for machine learning models
import seaborn as sns # visualization
import matplotlib.pyplot as plt # visualization

from sklearn.feature_extraction import DictVectorizer # Machine Learning
from sklearn.linear_model import LinearRegression # Machine Learning
from sklearn.linear_model import Lasso # Regularization
from sklearn.linear_model import Ridge # Regularization
from sklearn.preprocessing import OneHotEncoder

from sklearn.metrics import mean_squared_error # Loss Function

In [3]:
pd.__version__

'1.4.2'

Download the JAN and FEB 2023 dataset using wget


In [4]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet

--2024-05-27 00:18:53--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 52.85.39.153, 52.85.39.97, 52.85.39.117, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|52.85.39.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47673370 (45M) [application/x-www-form-urlencoded]
Saving to: ‘yellow_tripdata_2023-01.parquet.2’

yellow_tripdata_202 100%[===================>]  45.46M  97.0MB/s    in 0.5s    

2024-05-27 00:18:54 (97.0 MB/s) - ‘yellow_tripdata_2023-01.parquet.2’ saved [47673370/47673370]

--2024-05-27 00:18:54--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet
Reusing existing connection to d37ci6vzurychx.cloudfront.net:443.
HTTP request sent, awaiting response... 200 OK
Length: 47748012 (46M) [application/x-www-form-urlencoded]
Saving to: ‘yellow_tripdata_2023-02.parquet.2’

yellow_tri

### Read the data in dataframe 

In [7]:
yellow_jan= pd.read_parquet('yellow_tripdata_2023-01.parquet')

In [8]:
yellow_jan.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


### get the number of columns using info 
we have 19 columns in here for jan data  

In [9]:
yellow_jan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

## Create a duration column to calculate the duration of a ride in minutes.

Take a look at the datatype of this duration which is in datetime54

## What are Lambdas in python

```md
- In Python, a lambda function is a small anonymous function.  
- Lambda functions are useful when you need a simple function for a short period of time and don't want to define a full function using the def keyword.
- We can apply a lambda function to both the columns and rows of the Pandas data frame.

```

`https://www.geeksforgeeks.org/applying-lambda-functions-to-pandas-dataframe/`

In [10]:
yellow_jan['duration'] = yellow_jan.tpep_dropoff_datetime - yellow_jan.tpep_pickup_datetime

## convert to minutes as some data points are sometimes in day, use lambda function as td using the total_seconds function

yellow_jan.duration = yellow_jan.duration.apply(lambda td: td.total_seconds()/60)

## Select Variable for prediction 
```md

## I need to give information on what these variables

- We will be using the `PULocationID ` and the `DOLocationID` for this prediciton.

- However they are `int64` data type and we need a `strings(object)` to make it a categorical datatype.

- why covert to PU & DO categorical?, when trip distance is already numerical

``

In [11]:
## convert PU and DO into categorical variable
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']


yellow_jan[categorical] = yellow_jan[categorical].astype(str)



```md
Now the PU and DO dataype has been changed to objects(string) datatype

```

In [12]:
yellow_jan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 20 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           object        
 8   DOLocationID           object        
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

## check the distribution of the duration numerical variable and see how skewed it is.

```md
What is the standard deviation of the duration variable?

```

In [13]:
yellow_jan.describe()

,VendorID,passenger_count,trip_distance,RatecodeID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
count,3.066766e+06,2.995023e+06,3.066766e+06,2.995023e+06,3.066766e+06,3.066766e+06,3.066766e+06,3.066766e+06,3.066766e+06,3.066766e+06,3.066766e+06,3.066766e+06,2.995023e+06,2.995023e+06,3.066766e+06
mean,1.730215e+00,1.362532e+00,3.847342e+00,1.497440e+00,1.194483e+00,1.836707e+01,1.537842e+00,4.882900e-01,3.367941e+00,5.184907e-01,9.820847e-01,2.702038e+01,2.274231e+00,1.074086e-01,1.566900e+01
std,4.438480e-01,8.961200e-01,2.495838e+02,6.474767e+00,5.294762e-01,1.780782e+01,1.789592e+00,1.034641e-01,3.826759e+00,2.017579e+00,1.833529e-01,2.216359e+01,7.718454e-01,3.556511e-01,4.259435e+01
min,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,-9.000000e+02,-7.500000e+00,-5.000000e-01,-9.622000e+01,-6.500000e+01,-1.000000e+00,-7.510000e+02,-2.500000e+00,-1.250000e+00,-2.920000e+01
25%,1.000000e+00,1.000000e+00,1.060000e+00,1.000000e+00,1.000000e+00,8.600000e+00,0.000000e+00,5.000000e-01,1.000000e+00,0.000000e+00,1.000000e+00,1.540000e+01,2.500000e+00,0.000000e+00,7.116667e+00
50%,2.000000e+00,1.000000e+00,1.800000e+00,1.000000e+00,1.000000e+00,1.280000e+01,1.000000e+00,5.000000e-01,2.720000e+00,0.000000e+00,1.000000e+00,2.016000e+01,2.500000e+00,0.000000e+00,1.151667e+01
75%,2.000000e+00,1.000000e+00,3.330000e+00,1.000000e+00,1.000000e+00,2.050000e+01,2.500000e+00,5.000000e-01,4.200000e+00,0.000000e+00,1.000000e+00,2.870000e+01,2.500000e+00,0.000000e+00,1.830000e+01
max,2.000000e+00,9.000000e+00,2.589281e+05,9.900000e+01,4.000000e+00,1.160100e+03,1.250000e+01,5.316000e+01,3.808000e+02,1.969900e+02,1.000000e+00,1.169400e+03,2.500000e+00,1.250000e+00,1.002918e+04


The standard deviation of the duration variable is 42.59

## Checking for outliers 
```md
we dont want bias estimations in our model/predictions.Outliers need to be removed in the dataset

we need to check the distribution of the duration variable. 

There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

```

In [14]:
yellow_jan = yellow_jan[(yellow_jan.duration >= 1)& (yellow_jan.duration <= 60)]

## print the data

yellow_jan.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,8.433333
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,6.316667
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,12.750000
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,9.616667
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,10.833333


## What fraction of the records left after you dropped the outliers?
```md
This is a huge dataset actually 

```

`3009173/3066766 = 0.98 == 98% of data is still left`


### One-hot encoding 

```md 
Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

- Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
- Fit a dictionary vectorizer
- Get a feature matrix from it

- What's the dimensionality of this matrix (number of columns)?
```

```md
One-hot encoding is a technique used to convert categorical variables into a numerical representation that can be used by machine learning algorithms. It is a common preprocessing step in data preparation.

- In many machine learning algorithms, categorical variables cannot be directly used as input, as these algorithms typically operate on numerical data. 

- One-hot encoding transforms each categorical value into a binary vector, where each vector has a length equal to the number of unique categories in the variable. The vector contains all zeros except for a single one at the index corresponding to the category of the original value.

- One-hot encoding allows machine learning models to effectively capture categorical information and handle it as numerical input. However, it increases the dimensionality of the data, especially when dealing with variables with many unique categories. This can impact computational efficiency and model performance, especially in cases where the number of categories is large relative to the number of observations.

- To illustrate, let’s apply one-hot encoding to the pickup and dropoff location IDs. We’ll use only these two features for our model.

```
`reference: https://stephen137.github.io/posts/MLOps_Zoomcamp_Week_1/MLOps_Zoomcamp_Week_1.html`

In [1]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(handle_unknown='ignore', sparse= False)
subset_df = yellow_jan.iloc[:, [7, 8]]

ohetransform = ohe.fit_transform(subset_df[['PULocationID']])

ohetransform.head()

NameError: name 'yellow_jan' is not defined

In [43]:

array_lists = yellow_jan[categorical].to_numpy()
                         
# pd.DataFrame({"A": [1, 2], "B": [3, 4]}).to_numpy()

# array([[1, 3],
# [2, 4]])

array_lists.shape

(3009173, 2)

In [15]:
## turn categorical column into a list of dictionaries to use as parameters for Vectorizer
dict_lists = yellow_jan[categorical].to_dict(orient='records')

In [21]:
## Iterate over the dict and check out the data
for i in range(5):
    print(dict_lists[i])

{'PULocationID': '161', 'DOLocationID': '141'}
{'PULocationID': '43', 'DOLocationID': '237'}
{'PULocationID': '48', 'DOLocationID': '238'}
{'PULocationID': '138', 'DOLocationID': '7'}
{'PULocationID': '107', 'DOLocationID': '79'}


In [45]:
## onehotenconder accepts array list  
for item in array_lists[:5]:
        print(item)

['161' '141']
['43' '237']
['48' '238']
['138' '7']
['107' '79']


In [50]:
## convert data to numpy array
array_ohe = np.array(yellow_jan[categorical])
array_ohe

array([['161', '141'],
       ['43', '237'],
       ['48', '238'],
       ...,
       ['114', '239'],
       ['230', '79'],
       ['262', '143']], dtype=object)

In [56]:
## instantiate a dictionary vectorizer 
## The DictVectorizer takes a little bit of memory, instead I will use OneHotEnconder instead 
dv = DictVectorizer()

ohe = OneHotEncoder(handle_unknown='ignore')

In [41]:
## onehotenconder accepts array list  
for item in array_lists[:10]:
        print(item)

['161' '141']
['43' '237']
['48' '238']
['138' '7']
['107' '79']
['161' '137']
['239' '143']
['142' '200']
['164' '236']
['141' '107']


## OneHotEncoder data type

```md
It seems like your data X is a NumPy array containing pairs of values.
- If you want to use scikit-learn's fit_transform method with this data, you'll need to convert it to a format that scikit-learn expects,
- which is typically a 2D array-like object with shape (n_samples, n_features).
```

## Example of ohe

enc = OneHotEncoder(handle_unknown='ignore')
X = [['Male', 1], ['Female', 3], ['Female', 2]]
enc.fit(X)

In [ ]:
## Convert to 2D array 

## convert to int

## arr_reshaped = array.list

In [59]:
## fit the vectorizer and transform the data into a feature matrix
## fm = dv.fit_transform(dict_lists)
## ohe.fit_transform(array_lists, y=None)
ohe = ohe.fit(array_ohe)




## convert the feature matrix to an array
## fm_array = fm.toarray()
##ohe_array = ohe.toarray()


## print the dimensionality of the feature matrix
# dimensionality = fm_array.shape
# print("Dimensionality:", dimensionality)

ohe.categories_


[array(['1', '10', '100', '101', '102', '106', '107', '108', '109', '11',
        '111', '112', '113', '114', '115', '116', '117', '118', '119',
        '12', '120', '121', '122', '123', '124', '125', '126', '127',
        '128', '129', '13', '130', '131', '132', '133', '134', '135',
        '136', '137', '138', '139', '14', '140', '141', '142', '143',
        '144', '145', '146', '147', '148', '149', '15', '150', '151',
        '152', '153', '154', '155', '156', '157', '158', '159', '16',
        '160', '161', '162', '163', '164', '165', '166', '167', '168',
        '169', '17', '170', '171', '172', '173', '174', '175', '177',
        '178', '179', '18', '180', '181', '182', '183', '184', '185',
        '186', '187', '188', '189', '19', '190', '191', '192', '193',
        '194', '195', '196', '197', '198', '199', '2', '20', '200', '201',
        '202', '203', '205', '206', '207', '208', '209', '21', '210',
        '211', '212', '213', '214', '215', '216', '217', '218', '219',
        

In [ ]:
#fm.shape
ohe.shape

In [ ]:
## use OneHotEncoder
ohe = OneHotEncoder()

## fit the one hot encoder and transform the data into a feature matrix
fme = ohe.fit_transform(array_lists, y=None)

In [ ]:
fme

In [ ]:
## convert the feature matrix to an array

fme.shape

### customize pre-processing function

In [ ]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

### Training a model

Use the feature matrix from the previoud step to train a Linear Regression model

In [ ]:
# Convert categorical columns to a dictionary
train_dicts = yellow_jan_22[categorical].to_dict(orient='records')

# Instantiate a dictionary vectorizer
dv = DictVectorizer()

# Set up trainign set
X_train = dv.fit_transform(train_dicts)